In [1]:
import requests
#Импортируем необходимые библиотеки

# Задание №1

Наша задача – найти все произведения, связанные с подсолнухами. Выполните поиск по слову "sunflowers". Посмотрим на формат ответа и сохраним id полученных объектов в список.

In [2]:
IDs = 'sunflowers'
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q='+IDs)
res = r.json()

In [3]:
sunflowers = pd.DataFrame(res)

In [4]:
sunflowers['objectIDs']

0     436524
1     484935
2     437112
3     210191
4     431264
       ...  
85    437980
86    631808
87    834580
88    834585
89    834765
Name: objectIDs, Length: 90, dtype: int64

# Задание №2

Соберем информацию об некоторых объектах из вышеуказанного списка, а именно выполним запрос для объекта #437980:

In [5]:
OBJECT_ID = '437980'
r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/'+OBJECT_ID)
obj_test = r_test.json()

In [6]:
obj_test['title'],obj_test['GalleryNumber'],obj_test['culture'],obj_test['artistDisplayName'],obj_test['objectID'],obj_test['objectName'],obj_test['department'] 

('Cypresses',
 '825',
 '',
 'Vincent van Gogh',
 437980,
 'Painting',
 'European Paintings')

# Задание №3

Напишем цикл, с помощью которого будет собрана информация об объектах, которые были получены в Задании №1. Результаты запишем в датафрейм.

In [7]:
df = pd.DataFrame()

In [8]:
for object in sunflowers['objectIDs']:
    r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/'+str(object))
    all_obj = r_test.json()
    try:
        df_temp = pd.DataFrame({'objectID':[all_obj['objectID']],
                                'culture':[all_obj['culture']],
                                'title':[all_obj['title']],
                                'period':[all_obj['period']],
                                'department':[all_obj['department']]})
        df = pd.concat([df,df_temp], ignore_index=True)
    except:
        continue
    

In [9]:
df[:10] #выведем первые 10 строк

,objectID,culture,title,period,department
0,436524,,Sunflowers,,European Paintings
1,484935,,Sunflowers,,Modern and Contemporary Art
2,437112,,Bouquet of Sunflowers,,European Paintings
3,210191,,Vase with sunflowers,,European Sculpture and Decorative Arts
4,431264,,Sunflowers,,Drawings and Prints
5,397949,,Sunflowers,,Drawings and Prints
6,656530,,Sunflowers,,Drawings and Prints
7,480725,,Large Sunflowers,,Modern and Contemporary Art
8,486590,,Moroccan Sunflowers,,Modern and Contemporary Art
9,485308,,Sunflower,,Modern and Contemporary Art


In [10]:
df[df.culture=='Japan'] #проверим корректность датасета

,objectID,culture,title,period,department
28,57922,Japan,Incense box,Edo period (1615–1868),Asian Art


# Задание №4

Теперь попробуем построить более сложный запрос с помощью секции "Search"! На этот раз параметры необходимо передать через "&", например, departmentId (id департамента – 1) и q (ключевое слово – cat).

Выполним поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat" и запишем id объектов в cat_ids. Используем цикл из Задания 3, чтобы получить данные о первой 1000 объектов из списка.

In [11]:
r = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats = r.json()
cat_ids = pd.DataFrame(cats)

In [12]:
df2 = pd.DataFrame()

In [13]:
for object in cat_ids['objectIDs'][:1000]:
    r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/'+str(object))
    all_obj = r_test.json()
    try:
        df_temp = pd.DataFrame({'objectID':[all_obj['objectID']],
                                'period':[all_obj['period']],
                                'department':[all_obj['department']]})
        df2 = pd.concat([df2,df_temp], ignore_index=True)
    except:
        continue

In [14]:
df2.head()

,objectID,period,department
0,49698,,Asian Art
1,49470,Qing dynasty (1644–1911),Asian Art
2,53222,Muromachi period (1392–1573),Asian Art
3,36221,,Asian Art
4,60873,Song dynasty (960–1279) or later,Asian Art


In [15]:
df2.period.value_counts() #проверяем результат

Edo period (1615–1868)                                          435
                                                                123
Qing dynasty (1644–1911)                                         45
Muromachi period (1392–1573)                                     29
Eastern Javanese period                                          29
                                                               ... 
Malla period                                                      1
Ming dynasty (1368–1644), Jiajing mark and period (1522–66)       1
Qing dynasty (1644–1911), Kangxi mark and period (1662–1722)      1
Qing dynasty (1644–1911), Yongzheng period (1723–35)              1
late Edo (1615–1868)-early Meiji period (1868–1912)               1
Name: period, Length: 97, dtype: int64